## Group Assignment: Machine Learning 2

This year's Group Assignment will verse on predicting employee attrition. The dataset is available on Kaggle: [Employee Attrition competition](https://www.kaggle.com/competitions/playground-series-s3e3/data)

* The goal is to predict whether an employee will leave the company or not (`Attrition` column, binary classification).
* The dataset is artificially generated, but it is based on real data: [original data](https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset)
    * Here you can find what each feature represents and their possible values.
* You're given a training set and a test set. 
    * Perform your analysis, experiments and model selection on the training set, and don't touch the test set until you're ready to submit your predictions.
    * You can save some data from the training set to use as a validation set, but you should not use the test set for this purpose.
    * Once you're comfortable with the performance of your model on the training set, you can use the test set to get a final estimate of the performance of your model.
    * The CSV file that you will submit, should contain the predictions of your model on the test set. This means that the CSV should contain as many rows as the test set, and a single column with the predictions (0 or 1).

### 1. Rules

* You should work within your team. If I see any signs of cheating between teams, that's and immediate fail for both groups.
  * If you have any question, post it on Slack and I will answer it as soon as possible.
* The grade will be the same for all the members of the team.
* Everyone within the team should contribute and all of you will have to either present or answer questions during the presentation.
  * If somebody in the team is not collaborating, let me know as soon as possible since I will not accept any excuses at the end of the term
  * Help the others in the team to understand the code and the results, because they might be asked to explain it during the presentation
* It's fine to explore and learn from out there. 
  * I want to see that you are learning and that you are trying to improve your skills. However, you should not just copy-paste code from other sources -tell me where is it from and show me it's useful for your assignment-
  * If you just copy and paste, I will know and I will ask you about it, so be prepared.
* The final submission should use any of the algorithms that we've seen during the course, so no neural networks or similar. There's will be a time for that, but not now.


### 2. Submission

**1 ZIP file per group, named `submission_group_X.zip`, containing:**
  * A Jupyter Notebook with the code and the results, mandatory to include the names of the members of the team.
  * A PDF file with the presentation.
  * A CSV file with the predictions for the test set.
  * Failing to submit any of the above in the required conditions will result in a 0 for the Group Assignment.

### 3. Grade

The Group Assignment will weight 40% of the final grade. The grade will be based on the following criteria:
* **40%: PDF report and presentation**
    * The report should be done for an executive audience, so don't go too much into the technical details underneath the algorithms. 
    * It should cover a brief exploration of the data, the experiments done regarding feature engineering and the algorithms used, the results and the conclusions.
    * For the presentation, I will choose somebody from the team to present.
* **30%: Code in Jupyter notebook**
    * Comment why you're doing what you're doing.
    * Document your experiments and the results in the notebook.
    * Make sure that you do all your `import`s at the beginning of the notebook, so that I know what packages you're using. If your code doesn't run, I will not grade it fully.
* **30%: Answers to my questions during the presentation**
    * There will be questions about the data and its exploration, the experiments done with the features and with the algorithms, and obviously about the performance and results.

In [28]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from collections import Counter
import pandas_profiling
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.svm import SVC
import plotly.express as px
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

In [21]:
train = pd.read_csv('/Users/mac/Desktop/group_assignment_ML2/data/train.csv', sep = ",")
test = pd.read_csv('/Users/mac/Desktop/group_assignment_ML2/data/test.csv', sep = ",")

In [37]:
X= train.drop(["Attrition"],axis=1)
y = train["Attrition"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=99)

Sc = StandardScaler()
ohe = OneHotEncoder()
numeric_features = ["Age","DailyRate","DistanceFromHome","Education","HourlyRate","JobLevel","MonthlyIncome","MonthlyRate","NumCompaniesWorked","PercentSalaryHike",
                    "TotalWorkingYears","TrainingTimesLastYear","YearsAtCompany","YearsInCurrentRole","YearsSinceLastPromotion","YearsWithCurrManager"]
categorical_features = ["BusinessTravel","Gender","Department","MaritalStatus","EducationField","JobRole","OverTime","WorkLifeBalance"]
preprocess = ColumnTransformer(
    transformers=[
                ('num', Sc, numeric_features),
                ('cat', ohe, categorical_features)])

models=[LogisticRegression(solver="liblinear",max_iter=1000),RandomForestClassifier(),GradientBoostingClassifier(random_state=99,learning_rate=0.05,max_depth=2,n_estimators=200),xgb.XGBClassifier(random_state=99,learning_rate=0.05,max_depth=2,n_estimators=200)]


auc=[]

for x in models:
    model = Pipeline(steps=[('preprocess', preprocess),
                            ('model', x)])
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    auc.append(roc_auc)

model_names = ['Logistic Regression', 'Random Forest','Gradient Boosting',"XGBClassifier"]
scores= pd.DataFrame({'model_names': model_names,
                      "auc":auc})
scores

,model_names,auc
0,Logistic Regression,0.811115
1,Random Forest,0.842245
2,Gradient Boosting,0.865791
3,XGBClassifier,0.866999


In [23]:
y_pred = model.predict(X_test)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print("auc: ",roc_auc) 

auc:  0.8669991260089456


In [24]:
# Make predictions on the test data
y_pred_test = model.predict(test)
Counter(y_pred_test)

Counter({0: 1092, 1: 27})